# Pipeline test notebook

### Import requirements

In [33]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import time

from ImageProcessing import ImageProcessing
from StereoMatcher import StereoMatcher

%matplotlib ipympl

### Load calibrations and other data

In [34]:
imageProcessing = ImageProcessing()
imageProcessing.loadMonoCalibrationResults()
imageProcessing.loadStereoCalibration()
imageProcessing.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [35]:
stereoMatcher = StereoMatcher("SGBM", \
                vertical=True, createRightMatcher=False)
stereoMatcher.referenceDispToDepthMatrix(\
imageProcessing.dispToDepthMatrix)

imageProcessing.initUndistortRectifyMap()
#stereoMatcher.createDisparityWLSFilter()

Reading from data/parametersSGBM.json


### Loading images

In [36]:
path = "captures/testImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-7


### Select image pair and display

In [37]:
imageNumber = 5

imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

cv2.imshow("left/right", np.hstack([imageL, imageR]))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Convert to grayscale and undistort

In [38]:
imageProcessing.convertToGrayscale(imageL, imageR)
imageProcessing.undistortRectifyRemap(imageProcessing.grayImageL, \
                                        imageProcessing.grayImageR)

### View undistorted image

In [39]:
cv2.imshow("left/right undistort", np.hstack(\
    [imageProcessing.undistortImageL, imageProcessing.undistortImageL]))

cv2.imshow("horizontal epipolar", \
    imageProcessing.drawHorEpipolarLines(\
        imageProcessing.undistortImageL, imageProcessing.undistortImageR))

cv2.imshow("vertical epipolar", \
    imageProcessing.drawVertEpipolarLines(\
        imageProcessing.undistortImageL, imageProcessing.undistortImageR))
        
cv2.waitKey(0)
cv2.destroyAllWindows()

### Compute disparity map

In [40]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessing.undistortImageL, \
                grayImageR=imageProcessing.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
#stereoMatcher.applyWLSFilterDisparity()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

minDisparity: 9.0
maxDisparity: 38.8125


### View disparity map

In [41]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE), cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [42]:
focalLength = imageProcessing.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

9.0
38.8125
(640, 360)
610.7979
2634.066


### View depth map

In [43]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE), cmap="Greys")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute point cloud

In [61]:
points = cv2.reprojectImageTo3D(stereoMatcher.disparityMapL, \
                                stereoMatcher.dispToDepthMatrix)

# Reshaping to a list of 3D coordinates
pointCloud = points.reshape((points.shape[0]*points.shape[1],3))[0::16]\
                                                        .astype(np.int16)

print (points.shape)
print (pointCloud.shape)

print (points[0,0])
print (pointCloud[0])

(640, 360, 3)
(14400, 3)
[ -713.31146 -1168.7544   2763.0342 ]
[ -713 -1168  2763]


Filtering extreme points

In [62]:
pointCloud = pointCloud[pointCloud[:, 2]>pointCloud[:, 2].min()]
pointCloud = pointCloud[pointCloud[:, 2]<pointCloud[:, 2].max()]
pointCloud = pointCloud[pointCloud[:, 1]>pointCloud[:, 1].min()]
pointCloud = pointCloud[pointCloud[:, 1]<pointCloud[:, 1].max()]
pointCloud = pointCloud[pointCloud[:, 0]>pointCloud[:, 0].min()]
pointCloud = pointCloud[pointCloud[:, 0]<pointCloud[:, 0].max()]

print (pointCloud.shape)

(11671, 3)


Rotate point cloud to have y forward

In [63]:
pointRotMatrix = np.array([ [ 0,  0, -1],
                            [-1,  0,  0],
                            [ 0,  1,  0] ])

pointCloud = np.dot(pointCloud[:], pointRotMatrix)

print (pointCloud.shape)
print (pointCloud[0])

(11671, 3)
[309 731 141]


### View point cloud

In [64]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(pointCloud[:,0], pointCloud[:,1], pointCloud[:,2], s=1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_ylim(0,)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [65]:
# From the matplotlib documentation:
# Prepare some coordinates
x, y, z = np.indices((8, 8, 8))

# Draw cuboids in the top left and bottom right corners, and a link between them
voxels = (x < 3) & (y < 3) & (z < 3)

ax = plt.figure().add_subplot(projection='3d')
ax.voxels(voxels)#, edgecolor='k')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
voxelSize = 10 # mm
voxelRange = np.array(
  [[pointCloud[:, 0].min(), pointCloud[:, 1].min(), pointCloud[:, 2].min()],\
  [pointCloud[:, 0].max(), pointCloud[:, 1].max(), pointCloud[:, 2].max()]])\
  .astype(np.int16)

print(voxelRange, "range")

voxelUnits = (((voxelRange[1]-voxelRange[0]) - \
            (voxelRange[1]-voxelRange[0]) % voxelSize)/10).astype(np.int16)
print(voxelUnits, "units")

voxelCenter = (voxelRange[0] + (voxelRange[1]-voxelRange[0])/2 - (voxelRange[0] + (voxelRange[1]-voxelRange[0])/2)%10).astype(np.int16)
print(voxelCenter, "center")

[[-500  668 -317]
 [ 318 2260  412]] range
[ 81 159  72] units
[-100 1460   40] center


In [68]:
# Sampling
voxelSize = 50 # mm
iterations = 0
occupyThreshold = 5
occupancy = []
startTime = time.perf_counter()

while len(pointCloud>0):

    sampledPoint = pointCloud[0]

    samplingLimit = np.empty_like(sampledPoint)
    for n in range(len(sampledPoint)):
        samplingLimit[n]=(sampledPoint[n]//voxelSize)*voxelSize

    mask = np.ones(pointCloud.shape[0], dtype=bool)

    for n in range(len(sampledPoint)):
        maskLower = pointCloud[:,n]>=samplingLimit[n]
        maskUpper = pointCloud[:,n]<samplingLimit[n]+voxelSize
        mask = np.logical_and(mask, np.logical_and(maskLower, maskUpper))
    pointsInVoxel = pointCloud[mask]

    if len(pointsInVoxel)>occupyThreshold:
        voxelMidpoint = samplingLimit+voxelSize/2
        occupancy.append(voxelMidpoint)

    pointCloud = pointCloud[np.invert(mask)]

    iterations+=1

occupancy = np.array(occupancy, dtype=np.int16)
print("Time taken: {:.5f}".format(time.perf_counter()-startTime))

Time taken: 0.05073


In [51]:
print("Iterations:", iterations)
print(occupancy)
print("Voxels:", occupancy.shape[0])

Iterations: 756
[[ 325  725  125]
 [ 275  725   75]
 [ 325  725   75]
 ...
 [-425  925  125]
 [-425  975 -225]
 [-475 1025   75]]
Voxels: 350


In [52]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(occupancy[:,0], occupancy[:,1], occupancy[:,2])
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_ylim(0,)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
if False:
    x,y,z=np.indices(voxelUnits)*100

    voxels = (x<(occupancy[0,0]+voxelSize/2)) & (y<(occupancy[0,1]+voxelSize/2)) & (z<(occupancy[0,2]+voxelSize/2)) & (x>(occupancy[0,0]-voxelSize/2)) & (y>(occupancy[0,1]-voxelSize/2)) & (z>(occupancy[0,2]-voxelSize/2))

    for voxelMindpoint in occupancy:
        voxel = (x<(voxelMidpoint[0]+voxelSize/2)) & (y<(voxelMidpoint[1]+voxelSize/2)) & (z<(voxelMidpoint[2]+voxelSize/2)) & (x>(voxelMidpoint[0]-voxelSize/2)) & (y>(voxelMidpoint[1]-voxelSize/2)) & (z>(voxelMidpoint[2]-voxelSize/2))
        voxels = voxels | voxel

    ax=plt.figure().add_subplot(projection="3d")
    ax.voxels(voxels)

    plt.show()